# Project Milestone 2

In [2]:
import math
import json
import random
import numpy as np
import pandas as pd

## Dataset Import

In [3]:
# Glove
glove_df_train = pd.read_csv("../project_data/data/glove/glove.train.csv")
glove_df_test = pd.read_csv("../project_data/data/glove/glove.test.csv")
glove_df_eval = pd.read_csv("../project_data/data/glove/glove.eval.anon.csv")

# Add bias
glove_df_train["bias"] = 1
glove_df_test["bias"] = 1
glove_df_eval["bias"] = 1

# Replace label 0 with -1
glove_df_train.loc[glove_df_train["label"] == 0, "label"] = -1
glove_df_test.loc[glove_df_test["label"] == 0, "label"] = -1
glove_df_eval.loc[glove_df_eval["label"] == 0, "label"] = -1

glove_df_train

# Bag of words
bow_df_train = pd.read_csv("../project_data/data/bag-of-words/bow.train.csv")
bow_df_test = pd.read_csv("../project_data/data/bag-of-words/bow.test.csv")
bow_df_eval = pd.read_csv("../project_data/data/bag-of-words/bow.eval.anon.csv")

# Add bias
bow_df_train["bias"] = 1
bow_df_test["bias"] = 1
bow_df_eval["bias"] = 1

# Replace label 0 with -1
bow_df_train.loc[bow_df_train["label"] == 0, "label"] = -1
bow_df_test.loc[bow_df_test["label"] == 0, "label"] = -1
bow_df_eval.loc[bow_df_eval["label"] == 0, "label"] = -1


# Bag of words
tfidf_df_train = pd.read_csv("../project_data/data/tfidf/tfidf.train.csv")
tfidf_df_test = pd.read_csv("../project_data/data/tfidf/tfidf.test.csv")
tfidf_df_eval = pd.read_csv("../project_data/data/tfidf/tfidf.eval.anon.csv")

# Add bias
tfidf_df_train["bias"] = 1
tfidf_df_test["bias"] = 1
tfidf_df_eval["bias"] = 1

# Replace label 0 with -1
tfidf_df_train.loc[tfidf_df_train["label"] == 0, "label"] = -1
tfidf_df_test.loc[tfidf_df_test["label"] == 0, "label"] = -1
tfidf_df_eval.loc[tfidf_df_eval["label"] == 0, "label"] = -1

print("Glove Dataset")
glove_df_train

print("BOW Dataset")
bow_df_train

print("TFIDF Dataset")
tfidf_df_train

Glove Dataset
BOW Dataset
TFIDF Dataset


label        x0        x1        x2        x3        x4        x5  \
0          1  0.177057  0.063437  0.083603  0.407604  0.119183  0.096883   
1          1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
2         -1  0.000000  0.109398  0.000000  0.000000  0.000000  0.000000   
3          1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
4          1  0.000000  0.000000  0.044455  0.000000  0.000000  0.000000   
...      ...       ...       ...       ...       ...       ...       ...   
17495      1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
17496      1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
17497     -1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
17498     -1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   
17499     -1  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   

             x6        x7        x8  ...  x9991  x9992  x9993  x9994  x9995  \
0      0.047931  0.117364  0.046370  ...    0.0    0.0    0.0    0.0    0.0   
1      0.020774  0.000000  0.020098  ...    0.0    0.0    0.0    0.0    0.0   
2      0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
3      0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
4      0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
...         ...       ...       ...  ...    ...    ...    ...    ...    ...   
17495  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
17496  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
17497  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
17498  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   
17499  0.000000  0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0   

       x9996  x9997  x9998  x9999  bias  
0        0.0    0.0    0.0    0.0     1  
1        0.0    0.0    0.0    0.0     1  
2        0.0    0.0    0.0    0.0     1  
3        0.0    0.0    0.0    0.0     1  
4        0.0    0.0    0.0    0.0     1  
...      ...    ...    ...    ...   ...  
17495    0.0    0.0    0.0    0.0     1  
17496    0.0    0.0    0.0    0.0     1  
17497    0.0    0.0    0.0    0.0     1  
17498    0.0    0.0    0.0    0.0     1  
17499    0.0    0.0    0.0    0.0     1  

[17500 rows x 10002 columns]

## Common Functions

In [4]:
def get_max_key_by_value(map):
    max_key = ""
    max_val = float("-inf")

    for key, val in map.items():
        if val > max_val:
            max_val = val
            max_key = key

    # print("map: ", map, "max_key: ", max_key)
    return max_key


def initialize_weights(feature_count):
    rand_start = -0.01
    rand_end = 0.01
    random_number = random.uniform(rand_start, rand_end)

    weights = []  # All weights and bias should be same.
    for _ in range(feature_count):
        weights.append(random_number)

    return weights


def predict(weights, example):
    value = np.dot(weights, example)
    return 1 if value > 0 else -1


def test_accuracy(df, weights, store_eval=False):
    eval_list = []
    total = df.shape[0]
    correct_prediction = 0

    for _, row in df.iterrows():
        example = row.tolist()
        actual_label = example[0]  # y
        example = example[1:]  # x

        predicted_label = predict(weights, example)
        if predicted_label == actual_label:
            correct_prediction += 1

        if store_eval:
            eval_list.append(predicted_label)

    # print(f"Test accuracy. Correct Pred: {correct_prediction}, Total: {total}")
    return correct_prediction / total, eval_list


def export_prediction_to_csv(file_name, prediction_list):
    # Change back -1 to 0
    for index, p in enumerate(prediction_list):
        if p == -1:
            prediction_list[index] = 0

    df = pd.DataFrame(prediction_list)
    df.to_csv(file_name, index=True, index_label="example_id", header=["label"])


## Simple Perceptron

In [5]:
epochs = 20
learning_rates = [1, 0.1, 0.01, 0.001]


def simple_perceptron(df, learning_rate, weights):
    for _, row in df.iterrows():
        example = row.tolist()
        actual_label = example[0]  # y
        example = example[1:]  # x

        value = actual_label * (np.dot(weights, example))

        # update
        if value < 0:
            for index in range(len(weights)):
                # w = w + r * y * x
                weights[index] += learning_rate * actual_label * example[index]

    return weights


def simple_perceptron_setup(df_train, df_test):
    best_weights = []
    best_accuracy = 0.0
    best_learning_rate = 0.0

    initial_weights = initialize_weights(feature_count=df_train.shape[1] - 1)

    for learning_rate in learning_rates:
        weights = initial_weights[:]  # New learning rate start with same initial weights.
        print(f"\n\tLearning rate: {learning_rate}")

        for epoch in range(epochs):
            # Shuffle the whole data frame.
            df_train = df_train.sample(frac=1, random_state=1)

            weights = simple_perceptron(df=df_train, learning_rate=learning_rate, weights=weights)
            accuracy, _ = test_accuracy(df=df_test, weights=weights, store_eval=False)
            print(f"\t\tEpoch: {epoch + 1}, Accuracy: {accuracy}")

            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                best_weights = weights
                best_learning_rate = learning_rate

    return best_learning_rate, best_accuracy, best_weights

## Average Perceptron

In [6]:
epochs = 50
learning_rates = [1, 0.1, 0.01]


def average_perceptron(df, learning_rate, weights, avg_weights):
    for _, row in df.iterrows():
        example = row.tolist()
        actual_label = example[0]  # y
        example = example[1:]  # x

        # y(wT x + b)
        value = actual_label * (np.dot(weights, example))

        # update
        if value < 0:
            for index in range(len(weights)):
                # w = w + r * y * x
                weights[index] += learning_rate * actual_label * example[index]

        for index in range(len(weights)):
            avg_weights[index] += weights[index]

    return weights, avg_weights


def average_perceptron_setup(df_train, df_test):
    best_weights = []
    best_accuracy = 0.0
    best_learning_rate = 0.0

    initial_weights = initialize_weights(feature_count=df_train.shape[1] - 1)

    for learning_rate in learning_rates:
        weights = initial_weights[:]  # New learning rate start with same initial weights.
        avg_weights = initial_weights[:]  # Copy the list because list is mutable.
        print(f"\n\tLearning rate: {learning_rate}")

        for epoch in range(epochs):
            # Shuffle the whole data frame.
            df_train = df_train.sample(frac=1, random_state=1)

            weights, avg_weights = average_perceptron(
                df=df_train, learning_rate=learning_rate, weights=weights, avg_weights=avg_weights
            )
            accuracy, _ = test_accuracy(df=df_test, weights=avg_weights, store_eval=False)
            print(f"\t\tEpoch: {epoch + 1}, Accuracy: {accuracy}")

            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                best_weights = weights
                best_learning_rate = learning_rate

    return best_learning_rate, best_accuracy, best_weights

## Aggressive Perceptron

In [7]:
epochs = 20
learning_rates = [1, 0.1, 0.01, 0.001]

def aggressive_perceptron(df, margin, weights):
    for _, row in df.iterrows():
        example = row.tolist()
        actual_label = example[0]  # y
        example = example[1:]  # x

        # y(wT x + b)
        learning_rate = (margin - (actual_label * (np.dot(weights, example)))) / (np.dot(example, example) + 1)
        value = actual_label * (np.dot(weights, example))

        # update
        if value <= margin:
            for index in range(len(weights)):
                # w = w + r * y * x
                weights[index] += learning_rate * actual_label * example[index]

    return weights

def aggressive_perceptron_setup(df_train, df_test):
    best_weights = []
    best_margin = 0.0
    best_accuracy = 0.0

    initial_weights = initialize_weights(feature_count=df_train.shape[1] - 1)

    for margin in margins:
        weights = initial_weights[:]  # New learning rate start with same initial weights.
        print(f"\n\tMargin: {margin}")

        for epoch in range(epochs):
            # Shuffle the whole data frame.
            df_train = df_train.sample(frac=1, random_state=1)

            weights = aggressive_perceptron(df=df_train, margin=margin, weights=weights)
            accuracy, _ = test_accuracy(df=df_test, weights=weights, store_eval=False)
            print(f"\t\tEpoch: {epoch + 1}, Accuracy: {accuracy}")

            if accuracy >= best_accuracy:
                best_accuracy = accuracy
                best_weights = weights
                best_margin = margin

    return best_margin, best_accuracy, best_weights

## Training Setup & Results

To invoke one of more datasets on one or more perceptron.

In [8]:
dataset_setup_dict = {
    "Glove": {"Train": glove_df_train, "Test": glove_df_test, "Eval": glove_df_eval},
    "BOW": {"Train": bow_df_train, "Test": bow_df_test, "Eval": bow_df_eval},
    "TFIDF": {"Train": tfidf_df_train, "Test": tfidf_df_test, "Eval": tfidf_df_eval},
}

train_setup_dict = {
    "PerceptronMap": {
        "Simple": simple_perceptron_setup,
        "Average": average_perceptron_setup,
        "Aggressive": aggressive_perceptron_setup,
    },
    # "Datasets": ["Glove"],
    # "Datasets": ["BOW"],
    "Datasets": ["TFIDF"],
    # "Datasets": ["Glove", "BOW", "TFIDF"],
    # "PerceptronSetups": ["Simple", "Aggressive"],
    "PerceptronSetups": ["Average"],
    # "PerceptronSetups": ["Aggressive"],
    # "PerceptronSetups": ["Simple", "Average", "Aggressive"],
}

def export_weights(file_name, weights):
    weights = np.array2string(weights)
    np.save(f"weights/{file_name}.npy", weights)

def train_setup():
    for perceptron in train_setup_dict["PerceptronSetups"]:
        for dataset in train_setup_dict["Datasets"]:
            print(f"\n{perceptron} Perceptron: {dataset} Dataset\n")

            best_hyper_param, best_accuracy, best_weights = train_setup_dict["PerceptronMap"][perceptron](
                df_train=dataset_setup_dict[dataset]["Train"], df_test=dataset_setup_dict[dataset]["Test"]
            )
            label = f"{ perceptron.lower()}_perceptron_{dataset.lower()}"
            export_weights(label, np.array(best_weights))

            # Import weights from the file
            loaded_weights = np.load(f"weights/{label}.npy")

            print(f"\n\tBest parameter: {best_hyper_param} & test accuracy: ", best_accuracy)
            _, prediction_list = test_accuracy(
                df=dataset_setup_dict[dataset]["Eval"], weights=best_weights, store_eval=True
            )

            export_file_name = "{0}_perceptron_{1}_eval_dataset_prediction.csv".format(
                perceptron.lower(), dataset.lower()
            )
            export_prediction_to_csv(export_file_name, prediction_list)


train_setup()

KeyError: 'PerceptronSetups'